In [1]:
import sys
import os, os.path

sys.path.append(os.path.join(os.getcwd() ,'/modules'))
root_path = "C:/git/Springboard-Public/Capstone Project 2/"
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/My Drive/Capstone Project 2/"

print('Current Working Dir: ', os.getcwd())
print('Root Path: ', root_path)

# We need to set the working directory since we are using relative paths from various locations
if os.getcwd() != root_path:
  os.chdir(root_path)

Current Working Dir:  C:\git\Springboard-Public\Capstone Project 2\notebooks
Root Path:  C:/git/Springboard-Public/Capstone Project 2/


In [2]:
from modules.lib.StandardTraining import *
from modules.models.ResNet import ResNet_GrayScale_12_Out

from torchsummary import summary

%matplotlib inline

In [3]:
force_cpu = True
device = torch.device('cuda' if ~force_cpu and torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')

Working on device=cuda


In [4]:
class Nets:
    
    def __init__(self):
        self.cur_idx = 0
        self.nets = []
        layers = [18] # ResNet Laysers 18, 34, 50, 101, 152
        lrs = [1e-5]
        batch_sizes = [32]
        observation_min_counts = [None, 20, 50]

        combinations = [(l, lr, bs, m) for l in layers for lr in lrs for bs in batch_sizes for m in observation_min_counts] 

        for combination in combinations:
            layers, lr, batch_size, m = combination   
            name = 'ResNet' + str(layers) + f'\n    learning rate:{lr}\n    batch size:{batch_size}\n    min obs:{m}'
            self.nets.append((name, layers, lr, batch_size, m))
            
    def __iter__(self):
        for name, layers, lr, batch_size, m in self.nets:
            net = ResNet_GrayScale_12_Out(layers=layers)
            net = nn.DataParallel(net)
            net.to(device)
            yield ModelLoop.getConfigObject(name, net, 
                                            learning_rate=lr, 
                                            batch_size=batch_size,
                                            observation_min_count = m)
    
    def __len__(self):
        return len(self.nets)

In [5]:
modelLoop = ModelLoop(number_images=25000, 
                      default_batch_size=32, 
                      default_learning_rate = 1e-4,
                      num_epochs=4,
                      device=device, 
                      nets=Nets())

Total Oversampled Rows Added to Train: 2,699

Total Oversampled Rows Added to Train: 7,081



In [6]:
modelLoop.train()

▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆

ResNet18
    learning rate:1e-05
    batch size:32
    min obs:None

▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Number of Training Images: 19,986
Number of Validation Images: 5,014
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [1/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 209.1436         
Time of Completion: 2020-07-25 10:52:19.964518          
Training Time: 0:03:17.078451          
Validation Time: 0:00:30.960261
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [2/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 190.8829         
Time of Completion: 2020-07-25 10:55:37.443603          
Training Time: 0:02:51.545404          
Validation Time: 0:00:25.932678
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [3/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔

,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,2217,0.001804,0.333333,0.003589,0.745154,0.259945
4,Edema,4745,0.247418,0.595939,0.349665,0.781971,0.500624
5,Consolidation,1168,0.000000,0.000000,0.000000,0.671414,0.101453
7,Atelectasis,3184,0.000000,0.000000,0.000000,0.626018,0.224733
9,Pleural_Effusion,6863,0.498761,0.648295,0.563782,0.783124,0.629625



VALIDATION

▔▔▔▔▔▔▔


,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,554,0.003610,0.181818,0.007080,0.749596,0.262175
4,Edema,1183,0.311074,0.532562,0.392743,0.770900,0.479372
5,Consolidation,303,0.000000,0.000000,0.000000,0.642988,0.098854
7,Atelectasis,775,0.000000,0.000000,0.000000,0.624113,0.219786
9,Pleural_Effusion,1666,0.566026,0.617147,0.590482,0.793739,0.636624








▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆

ResNet18
    learning rate:1e-05
    batch size:32
    min obs:20

▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Number of Training Images: 22,672
Number of Validation Images: 5,027
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [1/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 261.2205         
Time of Completion: 2020-07-25 11:07:35.971870          
Training Time: 0:04:25.278869          
Validation Time: 0:00:29.190968
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [2/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 238.5352         
Time of Completion: 2020-07-25 11:12:30.471660          
Training Time: 0:04:25.482358          
Validation Time: 0:00:29.016435
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [3/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔

...

,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,525,0.005714,0.500000,0.011299,0.734268,0.240123
4,Edema,1186,0.172850,0.646688,0.272788,0.774304,0.492279
5,Consolidation,286,0.000000,0.000000,0.000000,0.651480,0.092623
7,Atelectasis,777,0.000000,0.000000,0.000000,0.605546,0.204310
9,Pleural_Effusion,1676,0.564439,0.617897,0.589959,0.781444,0.626336








▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆

ResNet18
    learning rate:1e-05
    batch size:32
    min obs:50

▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Number of Training Images: 27,045
Number of Validation Images: 5,036
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [1/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 328.2491         
Time of Completion: 2020-07-25 11:25:46.724285          
Training Time: 0:03:51.608896          
Validation Time: 0:00:26.174011
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [2/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 279.9226         
Time of Completion: 2020-07-25 11:30:04.978953          
Training Time: 0:03:51.839259          
Validation Time: 0:00:26.414412
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [3/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔

,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,4114,0.395965,0.912094,0.552203,0.875297,0.698340
4,Edema,6645,0.479910,0.727750,0.578398,0.867544,0.718915
5,Consolidation,3012,0.459163,0.987152,0.626785,0.860107,0.685564
7,Atelectasis,5473,0.331993,0.940963,0.490816,0.805310,0.642360
9,Pleural_Effusion,9882,0.643999,0.764904,0.699264,0.871228,0.814022



VALIDATION

▔▔▔▔▔▔▔


,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,568,0.073944,0.396226,0.124629,0.732814,0.255506
4,Edema,1179,0.526718,0.450000,0.485346,0.751751,0.455728
5,Consolidation,315,0.003175,0.333333,0.006289,0.625522,0.098396
7,Atelectasis,760,0.030263,0.232323,0.053551,0.573116,0.189285
9,Pleural_Effusion,1744,0.367546,0.644221,0.468054,0.754281,0.595919
